# Ejemplo de Agente de Reserva de Hotel y Vuelo

Esta solución te ayudará a reservar boletos de avión y hotel. El escenario es un viaje desde Londres Heathrow LHR el 20 de febrero de 2024 a Nueva York JFK, regresando el 27 de febrero de 2025, volando en clase económica solo con British Airways. Quiero alojarme en un hotel Hilton en Nueva York, por favor proporciona los costos del vuelo y del hotel.


# Inicializar el Servicio de Agente de Azure AI y obtener información de configuración desde **.env**

### **.env** 

Crea un archivo .env 

**.env** contiene la cadena de conexión del Servicio de Agente de Azure AI, el modelo utilizado por AOAI y el servicio correspondiente de API de Búsqueda de Google, ENDPOINT, etc.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "El nombre de implementación de tu modelo del Servicio de Agente de Azure AI"

[**NOTE**] Necesitarás un modelo con un límite de 100,000 tokens por minuto (Rate Limit) y un límite de 600 solicitudes por minuto (Request per minute).

  Puedes obtener el modelo en Azure AI Foundry - Model and Endpoint. 

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "La cadena de conexión de tu proyecto del Servicio de Agente de Azure AI"

  Puedes obtener la cadena de conexión del proyecto en la vista general de tu proyecto en la pantalla del portal de AI Foundry.

- **SERPAPI_SEARCH_API_KEY** = "Tu clave API de búsqueda de SERPAPI"
- **SERPAPI_SEARCH_ENDPOINT** = "Tu endpoint de búsqueda de SERPAPI"

Para obtener el Nombre de Implementación del Modelo y la Cadena de Conexión del Proyecto del Servicio de Agente de Azure AI, necesitas crear el Servicio de Agente de Azure AI. Se recomienda usar [esta plantilla](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) para crearlo directamente （***Nota:*** El Servicio de Agente de Azure AI actualmente está configurado en una región limitada. Se recomienda que consultes [este enlace](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) para configurar la región).

El agente necesita acceder a SERPAPI. Se recomienda registrarse usando [este enlace](https://serpapi.com/searches). Después de registrarte, puedes obtener una clave API y un ENDPOINT únicos.


# Configuración

Para ejecutar este cuaderno, debes asegurarte de haber instalado las bibliotecas necesarias ejecutando `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Tu versión de Semantic Kernel debería ser al menos 1.27.2.


Carga tu archivo .env de configuración y recursos, por favor asegúrate de haber añadido tus claves y configuraciones y de haber creado un archivo .env local.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Iniciar sesión en Azure

Ahora necesitas iniciar sesión en Azure. Abre una terminal y ejecuta el siguiente comando:

```bash
az login
```

Este comando te pedirá que ingreses tus credenciales de Azure, lo que permitirá que el servicio Azure AI Agent funcione correctamente.


# Explicación:
Esta es una variable que almacena la clave API para acceder a un servicio de API de SERP (Página de Resultados del Motor de Búsqueda). Una clave API es un identificador único que se utiliza para autenticar las solicitudes asociadas con tu cuenta.

Propósito: El propósito de esta línea es almacenar la clave API en una variable para que pueda ser utilizada para autenticar solicitudes al servicio de API de SERP. La clave API es necesaria para acceder al servicio y realizar búsquedas.

Cómo obtener una clave API de SERP: Para obtener una clave API de SERP, sigue estos pasos generales en https://serpapi.com (los pasos exactos pueden variar dependiendo del servicio específico de API de SERP que estés utilizando):

Elige un servicio de API de SERP: Hay varios servicios de API de SERP disponibles, como SerpAPI, Google Custom Search JSON API, entre otros. Elige el que mejor se adapte a tus necesidades.

Regístrate para obtener una cuenta: Ve al sitio web del servicio de API de SERP elegido y regístrate para obtener una cuenta. Es posible que necesites proporcionar información básica y verificar tu dirección de correo electrónico.

Crea una clave API: Después de registrarte, inicia sesión en tu cuenta y navega a la sección de API o al panel de control. Busca una opción para crear o generar una nueva clave API.  
Copia la clave API en tu archivo .env.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Explicación:
BASE_URL: Esta es una variable que almacena la URL base para el punto de acceso de la API SERP. El nombre de la variable BASE_URL es una convención utilizada para indicar que esta URL es el punto de partida para realizar solicitudes a la API.  
'https://serpapi.com/search':  

Esta es la cadena de URL real asignada a la variable BASE_URL. Representa el punto de acceso para realizar consultas de búsqueda utilizando la API SERP.

# Propósito:
El propósito de esta línea es definir una constante que contiene la URL base para la API SERP. Esta URL se utilizará como punto de partida para construir solicitudes a la API y realizar operaciones de búsqueda.

# Uso:
Al definir la URL base en una variable, puedes reutilizarla fácilmente en tu código siempre que necesites realizar solicitudes a la API SERP. Esto hace que tu código sea más fácil de mantener y reduce el riesgo de errores al evitar codificar la URL directamente en múltiples lugares. El ejemplo actual es https://serpapi.com/search?engine=bing, que utiliza la API de búsqueda de Bing. Se puede seleccionar una API diferente en https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Explicación:

Aquí es donde se encuentra el código de tu plugin.

Definición de Clase: `class BookingPlugin`: Define una clase llamada BookingPlugin que contiene métodos para reservar hoteles y vuelos.

Método de Reserva de Hotel:

- `@kernel_function(description="booking hotel")`: Un decorador que describe la función como una función kernel para reservar hoteles.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Define un método para reservar hoteles con parámetros anotados y un tipo de retorno.

El método construye un diccionario de parámetros para la solicitud de reserva de hotel y envía una solicitud GET a la API SERP. Verifica el estado de la respuesta y devuelve las propiedades del hotel si tiene éxito, o None si la solicitud falló.

Método de Reserva de Vuelo:

- `@kernel_function(description="booking flight")`: Un decorador que describe la función como una función kernel para reservar vuelos.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Define un método para reservar vuelos con parámetros anotados y un tipo de retorno.

El método construye diccionarios de parámetros para las solicitudes de vuelos de ida y vuelta, y envía solicitudes GET a la API SERP. Verifica el estado de la respuesta y agrega la información del vuelo a la cadena de resultados si tiene éxito, o imprime un mensaje de error si la solicitud falló. El método devuelve la cadena de resultados que contiene la información del vuelo.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Explicación:
Importar declaraciones: Importa los módulos necesarios para las credenciales de Azure, el agente de IA, el contenido de los mensajes del chat, el rol del autor y el decorador de funciones del kernel.

Administrador de contexto asíncrono: `async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,)`: Esto configura un administrador de contexto asíncrono para manejar las credenciales de Azure y crear un cliente para el agente de IA.

Nombre e instrucciones del agente: 
- `AGENT_NAME = "BookingAgent"`: Define el nombre del agente.
- `AGENT_INSTRUCTIONS = """..."""`: Proporciona instrucciones detalladas para el agente sobre cómo manejar solicitudes de reservas.

Crear definición del agente: `agent_definition = await client.agents.create_agent(...)`: Crea una definición del agente con el modelo, nombre e instrucciones especificados.

Crear agente de AzureAI: `agent = AzureAIAgent(...)`: Crea un agente de AzureAI utilizando el cliente, la definición del agente y el plugin definido.

Crear hilo: `thread: AzureAIAgentThread | None = None`: Crea un hilo para el agente. No es necesario crear un hilo primero; si se proporciona el valor `None`, se creará un nuevo hilo durante la primera invocación y se devolverá como parte de la respuesta.

Entradas del usuario: `user_inputs = ["..."]`: Define una lista de entradas del usuario para que el agente las procese.

En el bloque `finally`, elimina el hilo y el agente para liberar recursos.


# Autenticación

La clase `DefaultAzureCredential` forma parte del SDK de Azure para Python. Proporciona una forma predeterminada de autenticarse con los servicios de Azure. Intenta autenticarse utilizando varios métodos en un orden específico, como variables de entorno, identidad administrada y credenciales de Azure CLI.

Operaciones Asíncronas: El módulo aio indica que la clase DefaultAzureCredential admite operaciones asíncronas. Esto significa que puedes usarla con asyncio para realizar solicitudes de autenticación sin bloqueo.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Aunque nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
